<a href="https://colab.research.google.com/github/ArkanDash/Advanced-RVC-Inference/blob/master/Advanced-RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RVC Inference

A modular Retrieval-based Voice Conversion framework with Gradio UI, training capabilities, and audio processing tools

[Discord](https://discord.gg/hvmsukmBHE) - [Github](https://github.com/ArkanDash/Advanced-RVC-Inference)

In [ ]:
# @title Mount Drive
from google.colab import drive
from google.colab._message import MessageError

try:
  drive.mount("/content/drive")
except MessageError:
  print("❌ Failed to mount drive")

In [ ]:
# @title Setup runtime environment
# @markdown ### Choose installation method:
install_method = "local"  # @param ["local", "git+uv", "git+pip"]
from IPython.display import clear_output
import os

# Install system dependencies for audio processing
!apt-get -y install libportaudio2 ffmpeg git -qq > /dev/null 2>&1

# Install uv for faster package management
!pip install uv -q

match install_method:
LOGS_PATH = "/content/Advanced-RVC-Inference/advanced_rvc_inference/assets/logs"
BACKUPS_PATH = f"/content/drive/MyDrive/RVCBackup"


  case 'local':
    # Clone from git and install locally (Recommended for development)
    if os.path.exists("Advanced-RVC-Inference"):
        %cd Advanced-RVC-Inference
        !git pull origin master
    else:
        !git clone https://github.com/ArkanDash/Advanced-RVC-Inference.git
        %cd Advanced-RVC-Inference

    # Install from requirements.txt
    !uv pip install -r requirements.txt --system -q

    # Install package in editable mode
    !uv pip install -e . --system -q

  case 'git+uv':
    # Install latest version from GitHub with uv (fastest)
    !uv pip install git+https://github.com/ArkanDash/Advanced-RVC-Inference.git -q

  case 'git+pip':
    # Install latest version from GitHub with pip
    !pip install git+https://github.com/ArkanDash/Advanced-RVC-Inference.git -q

clear_output()
print("Finished installing Advanced RVC Inference!")
print("Use the cell below to start the server.")

In [ ]:
# @title Sync with Google Drive
# @markdown 💾 Run this cell to automatically Save/Load models from your mounted drive
# @title
# @markdown This will merge and link your `RVCBackup` folder from gdrive to this notebook
from IPython.display import display, clear_output
from pathlib import Path

non_bak_folders = ["mute", "reference", "zips", "mute_spin", "mute_spin-v2"]
non_bak_path = "/tmp/rvc_logs"


def press_button(button):
  button.disabled = True


def get_date(path: Path):
  from datetime import datetime
  return datetime.fromtimestamp(int(path.stat().st_mtime))


def get_size(path: Path):
  !du -shx --apparent-size "{path}" > /tmp/size.txt
  return open("/tmp/size.txt").readlines().pop(0).split("   ")[0] + "B"


def sync_folders(folder: Path, backup: Path):
  from ipywidgets import widgets
  from jupyter_ui_poll import ui_events
  from time import sleep

  local = widgets.VBox([
      widgets.Label(f"Local: {LOGS_PATH.removeprefix('/content/')}/{folder.name}/"),
      widgets.Label(f"Size: {get_size(folder)}"),
      widgets.Label(f"Last modified: {get_date(folder)}")
  ])
  remote = widgets.VBox([
      widgets.Label(f"Remote: {BACKUPS_PATH.removeprefix('/content/')}/{backup.name}/"),
      widgets.Label(f"Size: {get_size(backup)}"),
      widgets.Label(f"Last modified: {get_date(backup)}")
  ])
  separator = widgets.VBox([
      widgets.Label("|||"),
      widgets.Label("|||"),
      widgets.Label("|||")
  ])
  radio = widgets.RadioButtons(
      options=[
          "Save local model to drive",
          "Keep remote model"
      ]
  )
  button = widgets.Button(
      description="Sync",
      icon="upload",
      tooltip="Sync model"
  )
  button.on_click(press_button)

  clear_output()
  print(f"Your local model '{folder.name}' is in conflict with it's copy in Google Drive.")
  print("Please select which one you want to keep:")
  display(widgets.Box([local, separator, remote]))
  display(radio)
  display(button)

  with ui_events() as poll:
    while not button.disabled:
      poll(10)
      sleep(0.1)

  match radio.value:
    case "Save local model to drive":
      !rm -r "{backup}"
      !mv "{folder}" "{backup}"
    case "Keep remote model":
      !rm -r "{folder}"


if Path("/content/drive").is_mount():
  !mkdir -p "{BACKUPS_PATH}"
  !mkdir -p "{non_bak_path}"

  if not Path(LOGS_PATH).is_symlink():
    for folder in non_bak_folders:
      folder = Path(f"{LOGS_PATH}/{folder}")
      backup = Path(f"{BACKUPS_PATH}/{folder.name}")

      !mkdir -p "{folder}"
      !mv "{folder}" "{non_bak_path}" &> /dev/null
      !rm -rf "{folder}"
      folder = Path(f"{non_bak_path}/{folder.name}")
      if backup.exists() and backup.resolve() != folder.resolve():
        !rm -r "{backup}"
      !ln -s "{folder}" "{backup}" &> /dev/null

    for model in Path(LOGS_PATH).iterdir():
      if model.is_dir() and not model.is_symlink():
        backup = Path(f"{BACKUPS_PATH}/{model.name}")

        if model.name == ".ipynb_checkpoints":
          continue

        if backup.exists() and backup.is_dir():
          sync_folders(model, backup)
        else:
          !rm "{backup}"
          !mv "{model}" "{backup}"

    !rm -r "{LOGS_PATH}"
    !ln -s "{BACKUPS_PATH}" "{LOGS_PATH}"

    clear_output()
    print("✅ Models are synced!")

  else:
    !rm "{LOGS_PATH}"
    !ln -s "{BACKUPS_PATH}" "{LOGS_PATH}"
    clear_output()
    print("✅ Models already synced!")

else:
  print("❌ Drive is not mounted, skipping model syncing")
  print("To sync your models, first mount your Google Drive and re-run this cell")

In [ ]:
# @title **Start server**
# @markdown ### Choose a sharing method:
method = "gradio"  # @param ["gradio", "localtunnel", "ngrok"]
ngrok_token = ""  # @param {type:"string"}
from IPython.display import clear_output
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Change to project directory if using local installation
if install_method == 'local':
    repo_dir = "Advanced-RVC-Inference"
    # Check if we're already in the repo directory
    if os.path.basename(os.getcwd()) != repo_dir:
        if os.path.exists(repo_dir):
            %cd $repo_dir
            !git pull -q

clear_output()

match method:

  case 'gradio':
    !rvc-gui --share

  case 'localtunnel':
    !pip install localtunnel -q
    !echo "Password IP: $(curl --silent https://ipv4.icanhazip.com)"
    !echo
    !lt --port 7869 & rvc-gui --listen --server-name 0.0.0.0

  case 'ngrok':
    !pip install pyngrok -q
    from pyngrok import ngrok
    ngrok.kill()
    if ngrok_token:
        ngrok.set_auth_token(ngrok_token)
    public_url = ngrok.connect(7869)
    print(f"Ngrok URL: {public_url}")
    !rvc-gui --listen --server-name 0.0.0.0 --server-port 7869